# Step 1: Best Contour Extraction/Prediction

In [62]:
orig_dir = 'CV-Assignment-Dataset'   #Test Images Directory
dest_dir = 'plot'                    #images stored with ROI boxes

In [63]:
from image_process import (
    threshold,
    morphology,
    find_contours,
    extra_contour_elimination,
    crop_contour,
    req_contour
)
import csv
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Load saved model after training

In [64]:
from keras.models import load_model
model = load_model('model.hdf5')

Plot and predict the required contour and also return the bounding box coordinates

In [65]:
def plot(filename,file,target):
    img = cv2.imread(filename)
    clone = img.copy()
    thresh = threshold(img)
    img, lines = find_contours(thresh)
    final_lines = extra_contour_elimination(lines)
    req = []
    for i,line in enumerate(final_lines):
        L = crop_contour(line,thresh)
        img = cv2.resize(L,(28,28))
        data = img.reshape(1,28,28,1)
        model_out = model.predict([data])

        if np.argmax(model_out)==1:
            str_label = '0'
        else:
            str_label = '1'

        if str_label=='1':
            print(file)
            req.append(i)
            name = dest_dir+'/'+file
            rect = cv2.minAreaRect(line)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            cv2.drawContours(clone,[box],0,(0,255,0),2)
            xmin = min(box[:,0])
            xmax = max(box[:,0])
            ymin = min(box[:,1])
            ymax = max(box[:,1])
            crop = [xmin,xmax,ymin,ymax]
            target.append([file,thresh,line,crop])
            cv2.imwrite(name,clone)
    return target

In [71]:
target = []
for file in os.listdir(orig_dir):
    path = os.path.join(orig_dir,file)
    target = plot(path,file,target)
boundary = target[0][3]

DSC08876.JPG
DJI_0422.JPG
DJI_0422.JPG
DJI_0083.JPG
DJI_0083.JPG
DSC01453.JPG
M1_F1.3_0405.JPG
M1_F1.3_0405.JPG
DJI_0617.JPG
DJI_0036.JPG
DJI_0616.JPG
DJI_0086.JPG
DJI_0086.JPG
DJI_0086.JPG
DJI_0086.JPG


# Step 2: Point of L extraction

Extraction using Center of mass, since the required coordinates lie near COM of L

In [82]:
def get_com(contours):

    kpCnt = len(contours)

    x = 0
    y = 0
    
    for kp in contours:
        x = x+kp[0][0]
        y = y+kp[0][1]


    com = np.array([x/kpCnt,y/kpCnt])
    return com

Check if the contour coordinates lie in the required bounding box coordinates

In [73]:
def check(rect,pt):
    logic = rect[0] < pt[0] < rect[1] and rect[2] < pt[1] < rect[3]
    return logic

Detect corner with Haris Corner detector.
Here, cv2.cornerSubPix() further refines the corners detected with sub-pixel accuracy. 

In [75]:
def get_gcp(gray,com,boundary):
    gray = np.float32(gray)
    dst = cv2.cornerHarris(gray,2,3,0.04)
    dst = cv2.dilate(dst,None)
    ret, dst = cv2.threshold(dst,0.001*dst.max(),255,0)
    dst = np.uint8(dst)

    # find centroids
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)


    # define the criteria to stop and refine the corners
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(gray,np.float32(centroids),(3,3),(-1,-1),criteria)
    dist_min = 1000000
    
    # Extra contour elimination
    for i,corner in enumerate(corners):
        ans = check(boundary,corner)
        if not ans:
            np.delete(corners,i)
            
    #check of eucliedien distance of COM from Harris SubPix corners 
    for corner in corners:
        euc_dist = np.linalg.norm(corner - com)
        if euc_dist<dist_min:
            dist_min = euc_dist
            gcp = corner
    return gcp

In [76]:
gcps = []
for target in target:
    com = get_com(target[2])
    boundary = target[3]
    file = target[0]
    gcps.append([file,gcp])
    print(file," ",gcp)

DSC08876.JPG   [4164.902    171.14754]
DJI_0422.JPG   [2609.2104 2253.8484]
DJI_0422.JPG   [ 507.13623 1613.8837 ]
DJI_0083.JPG   [2458.4124 2062.5876]
DJI_0083.JPG   [2335.0178   573.03754]
DSC01453.JPG   [3128.804  1170.9548]
M1_F1.3_0405.JPG   [ 982.9792 2688.448 ]
M1_F1.3_0405.JPG   [ 665.07776 1486.0809 ]
DJI_0617.JPG   [3780.3523     33.220127]
DJI_0036.JPG   [1742.4098 2597.1355]
DJI_0616.JPG   [ 254.71074 1455.9338 ]
DJI_0086.JPG   [2528.279  2445.4182]
DJI_0086.JPG   [1474.6449 1496.343 ]
DJI_0086.JPG   [2360.0862   777.64453]
DJI_0086.JPG   [2284.9143   124.52857]


# Record CSV Output

In [77]:
import csv
import os

In [80]:
file = os.path.join(os.getcwd(),'output.csv')
print(file)

/home/vishnoitanuj/Skylark_CV/output.csv


In [81]:
with open(file,'w') as f:
    writer = csv.writer(f)
    writer.writerow(["FileName", "GCPLocation"])
    for i in range(len(gcps)):
        row = []
        img = gcps[i][0]
        row.append(img)
        points = '['
        points += str(gcps[i][1])
        for j in range(i+1,len(gcps)):
            if gcps[j][0]==img:
                points += str(gcps[j][1])
                i += 1
            else:
                break
        points += ']'
        row.append(points)
        writer.writerow(row)
    f.close
        
                